In [1]:
import pandas as pd
import numpy as np 
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import date
import time
plt.style.use('seaborn')
%matplotlib inline

In [2]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import leaguegamefinder
from nba_api.stats.endpoints import leaguedashoppptshot
from sklearn.linear_model import Ridge
from sklearn.linear_model import LinearRegression as LR

In [3]:
nba_teams = teams.get_teams()
nba_teams[0]

{'id': 1610612737,
 'full_name': 'Atlanta Hawks',
 'abbreviation': 'ATL',
 'nickname': 'Hawks',
 'city': 'Atlanta',
 'state': 'Atlanta',
 'year_founded': 1949}

currentSeason is the year of which the NBA season starts. For ex: if the current season is the 2019-2020 then currentSeason value would be 2019.

In [4]:
today = date.today()

if today.month > 9:
    currentSeason = str(today.year)
else: 
    currentSeason = str(today.year - 1)

## extract_data function

This function takes in the names of the home & away team given by the user & fitlers out their games from the current season.

The function will extract the offensive & defensive data according to the games played by the teams. Since the algorithm is based on the assumption that the game will be held on either team's home floor, this function will filter out statistics according to their home & away games.

For example: Mavericks @ Clippers. The function will extract offensive & defensive statistics from the Mavericks' away games and from the Clippers' home games.

In [10]:
def extract_data(currentSeason):
    nba_teams = teams.get_teams()
    
    homeTeam = str(input("Name of the home team: "))
    awayTeam = str(input("Name of the away team: "))
    
    homeData = [team for team in nba_teams if team['nickname'] == homeTeam][0]
    homeID = homeData['id']

    awayData = [team for team in nba_teams if team['nickname'] == awayTeam][0]
    awayID = awayData['id']
    
    # Query for offensive stats of teams desired
    home_gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable = homeID)
    away_gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable = awayID)

    # The first DataFrame of those returned is what we want.
    homeTeam_games = home_gamefinder.get_data_frames()[0]
    awayTeam_games = away_gamefinder.get_data_frames()[0]

    offense = [homeTeam_games, awayTeam_games]

    currentSeason_games = []

    for i in offense: 
        thisSeason_games = i[i.SEASON_ID.str[-4:] == currentSeason]
        currentSeason_games.append(thisSeason_games)
        
    homeTeam_games = currentSeason_games[0][currentSeason_games[0].MATCHUP.str.contains('vs.')]
    awayTeam_games = currentSeason_games[1][currentSeason_games[1].MATCHUP.str.contains('@')]
    
    offense_categories = ['FGM', 'FGA', 'FG3M', 'FG3A', 'FTM', 'FTA']   
    
    homeOffense = homeTeam_games[offense_categories]
    awayOffense = awayTeam_games[offense_categories]
    
    # Query for defensive stats of teams desired
    home_defense = leaguedashoppptshot.LeagueDashOppPtShot(team_id_nullable = homeID, location_nullable = "Home")
    away_defense = leaguedashoppptshot.LeagueDashOppPtShot(team_id_nullable = awayID, location_nullable = "Road")

    homeTeam_defense = home_defense.get_data_frames()
    awayTeam_defense = away_defense.get_data_frames()

    defense = [homeTeam_defense[0], awayTeam_defense[0]]
    
    defense_categories = ['G','FG2A', 'FG3A']
    
    homeDefense = defense[0][defense_categories]
    awayDefense = defense[1][defense_categories]
    
    return homeOffense, awayOffense, homeDefense, awayDefense, homeTeam, awayTeam

In [11]:
def offense(table):
    fgm = table.iloc[:,0].values
    fga = table.iloc[:,1].values
    tpm = table.iloc[:,2].values
    tpa = table.iloc[:,3].values
    ftm = table.iloc[:,4].values
    fta = table.iloc[:,5].values
    
    avg_fta = int(np.mean(fta))
    fgm = fgm - tpm
    fga = fga - tpa
    
    return fgm, fga, tpm, tpa, ftm, fta, avg_fta

def defense(table):
    games = int(table.iloc[:,0])
    avg_2fga = int(table['FG2A']/games)
    avg_3fga = int(table['FG3A']/games)
    return avg_2fga, avg_3fga

In [12]:
def linreg(attempts, makes, expected_attempts):
    
    new_attempts = attempts.reshape(-1,1)
    new_makes = makes.reshape(-1,1)
    
    new_expected_attempts = np.array([expected_attempts]).reshape(-1,1)
    
    ridge = LR()
    ridge.fit(new_attempts, new_makes)
    expected_makes = ridge.predict(new_expected_attempts)
    
    return expected_makes

def calc_score(fg,three,ft):
    score = fg*2 + three*3 + ft
    return int(score)

In [13]:
def main():
    homeOffense, awayOffense, homeDefense, awayDefense, homeTeam, awayTeam = extract_data(currentSeason)
    
    fgm1, fga1, tpm1, tpa1, ftm1, fta1, avg_fta1 = offense(homeOffense)
    fgm2, fga2, tpm2, tpa2, ftm2, fta2, avg_fta2 = offense(awayOffense)
    
    avg_fga1, avg_3fga1 = defense(homeDefense)
    avg_fga2, avg_3fga2 = defense(awayDefense)
    
    home_e2fgm = linreg(fga1, fgm1, avg_fga2)
    home_e3fgm = linreg(tpa1, tpm1, avg_3fga2)
    home_eftm = linreg(fta1, ftm1, avg_fta1)
    
    away_e2fgm = linreg(fga2, fgm2, avg_fga1)
    away_e3fgm = linreg(tpa2, tpm2, avg_3fga1)
    away_eftm = linreg(fta2, ftm2, avg_fta2)
    
    homeScore = calc_score(home_e2fgm, home_e3fgm, home_eftm)
    awayScore = calc_score(away_e2fgm, away_e3fgm, away_eftm)
    
    localtime = time.asctime(time.localtime(time.time()))
    
    if homeScore == awayScore:
        print("\nThe current time is", localtime)
        print("The game may go into overtime.")
        print("\n{}: {} \n{}: {}".format(homeTeam, homeScore, awayTeam, awayScore))
    
    else:
        print("\nThe current time is", localtime)
        print("\n{}: {} \n{}: {}".format(homeTeam, homeScore, awayTeam, awayScore))

In [9]:
main()

Name of the home team: Celtics
Name of the away team: Heat

The current time is Thu Sep 24 23:44:29 2020

Celtics: 109 
Heat: 110
